In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install Sastrawi
!pip install translate
!pip install tweet-preprocessor
!pip install textblob
!pip install wordcloud
!pip install nltk
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
import preprocessor as p
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data Cleaning

In [ ]:
df = pd.read_csv('/content/prabowo.csv')
df = df[['full_text', 'username', 'created_at']]
df

,full_text,username,created_at
0,"@andre_rosiade eh botak, bilang ke capres lu d...",Yhafidzun15,Fri Dec 15 04:10:46 +0000 2023
1,"Elektabilitas tembus 45 persen, AHY: Prabowo b...",Iza_Zot,Fri Dec 15 04:10:46 +0000 2023
2,"Beri pujian ke AHY, Prabowo: Pemimpin yang heb...",Iza_Zot,Fri Dec 15 04:10:43 +0000 2023
3,@kurawa jokowi hebat....setuju tapi melihat pe...,romobing,Fri Dec 15 04:10:41 +0000 2023
4,Prabowo prediksi AHY akan jadi pemimpin di tin...,Iza_Zot,Fri Dec 15 04:10:38 +0000 2023
...,...,...,...
498,"@kompascom Memang tidak mudah, sekarang masyar...",HAPOSANM,Fri Dec 15 03:56:02 +0000 2023
499,Visi Prabowo - Gibran membawa harapan baru bag...,istimnatiara,Fri Dec 15 03:56:01 +0000 2023
500,@supersaiyaindo @ch_chotimah2 Pembenaran akan ...,DylanBr03197352,Fri Dec 15 03:56:01 +0000 2023
501,Prabowo Gibran berkomitmen untuk mengoptimalka...,clara144495,Fri Dec 15 03:56:00 +0000 2023


In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.dropna()
df.isnull().sum()

full_text     0
username      0
created_at    0
dtype: int64

In [ ]:
def cleanTwitter(text):
  text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
  text = re.sub(r'#\w+', ' ', text)
  text = re.sub(r'RT[\s]+', ' ', text)
  text = re.sub(r'https?://\S+', ' ', text)
  text = re.sub(r'[^A-Za-z0-9 ]', ' ', text)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

df['full_text'] = df['full_text'].apply(cleanTwitter)
df['full_text'] = df['full_text'].str.lower()

In [ ]:
df = df.drop_duplicates(subset=['full_text'])

In [ ]:
df

,full_text,username,created_at
0,eh botak bilang ke capres lu dulu dia ngobrol ...,Yhafidzun15,Fri Dec 15 04:10:46 +0000 2023
1,elektabilitas tembus 45 persen ahy prabowo bis...,Iza_Zot,Fri Dec 15 04:10:46 +0000 2023
2,beri pujian ke ahy prabowo pemimpin yang hebat...,Iza_Zot,Fri Dec 15 04:10:43 +0000 2023
3,jokowi hebat setuju tapi melihat penampilan pr...,romobing,Fri Dec 15 04:10:41 +0000 2023
4,prabowo prediksi ahy akan jadi pemimpin di tin...,Iza_Zot,Fri Dec 15 04:10:38 +0000 2023
...,...,...,...
498,memang tidak mudah sekarang masyarakat mau tau...,HAPOSANM,Fri Dec 15 03:56:02 +0000 2023
499,visi prabowo gibran membawa harapan baru bagi ...,istimnatiara,Fri Dec 15 03:56:01 +0000 2023
500,pembenaran akan selalu ada tapi jika anda mau ...,DylanBr03197352,Fri Dec 15 03:56:01 +0000 2023
501,prabowo gibran berkomitmen untuk mengoptimalka...,clara144495,Fri Dec 15 03:56:00 +0000 2023


# Preprocessing

In [ ]:
# Set the display option
# pd.set_option('display.max_colwidth', None)

for i, text in enumerate(df['full_text']):
  print(f'{i}. {text}')

0. eh botak bilang ke capres lu dulu dia ngobrol ama anies bahas tentang komitmen dia untuk nuntasin dibjakarta selama dia ngejabat dan gamau nyalonin dalam bentuk apapun di pilpres 2019 karenaaa udh komitmen ama pak prabowo mau fokus di jakartaaa dulu jadi gubernur gemes juga gw
1. elektabilitas tembus 45 persen ahy prabowo bisa menang satu putaran mua agusyudhoyono pdemokrat demokrat bareng ahy
2. beri pujian ke ahy prabowo pemimpin yang hebat untuk bangsa indonesia muj pdemokrat agusyudhoyono demokrat bareng ahy
3. jokowi hebat setuju tapi melihat penampilan prabowo di debat 1 beliau salah pilih orang
4. prabowo prediksi ahy akan jadi pemimpin di tingkat nasional fc agusyudhoyono pdemokrat demokrat bareng ahy
5. acara hari ini seru banget prabowo sby djoko suyanto dan sutanto dari alumni akabri 1970 1973 hadir semua dekade08
6. warga indonesia butuh sosok pemimpin yang seperti mas gibran adanya
7. demokrat sumsel siap menangkan pasangan prabowo gibran di pilpres 2024 vh pdemokrat ag

In [ ]:
#normalisasi

norm = {
    "gibran": "rekan prabowo",
    "pragibs": "prabowo",
    "pragib": "prabowo",
    "mahfud": "",
    "mahamin" : "",
    " ri ": " indonesia ",
    " gak ": " tidak ",
    " yg ": " yang ",
    " jg ": " juga ",
    " tlh ": " telah ",
    " ttp ": " tetap ",
    " donk ": " ",
    " tp ": " tetapi ",
    " ud ": " sudah ",
    " lu ": " anda ",
    " ga ": " tidak ",
    " dg ": " dengan ",
    " dgn ": " dengan ",
    " sdh ": " sudah ",
    " tdk ": " tidak ",
    " tak ": " tidak ",
    " g ": " tidak ",
    " msk ": " masuk ",
    " ngomong ": " mengatakan ",
    "gw": "saya",
    "gua": "saya",
    " tak ": " tidak ",
    " anies ": " ",
    " anis ": " ",
    " ganjar ": " ",
    "siang": "",
    "selamat natal": "",
    "christmas": "",
    "home alone": "",
    "lembang": "",
    "miskha": "",
    "fuji": "",
    "istj": "",
    "wa aja": "",
    "palestine": "",
    "pukis": "",
    "morowali": "",
    "choi minho": "",
    "drmn": "darimana",
    " kl ": " jikalau ",
    " jd ": " jadi ",
    " kmrn ": " kemarin ",
    " msk ": " masuk ",
    " sm ": " sama ",
    " nkri ": " Indonesia ",
    " planga plongo ": " bodoh ",
    " dungu ": " bodoh ",
    " kotorveesi ": " kontroversi ",
    " sj ": " saja "

}


def normalisasi(text):
  for i in norm:
    text = text.replace(i, norm[i])
  return text

df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
df

,full_text,username,created_at
0,eh botak bilang ke capres anda dulu dia ngobro...,Yhafidzun15,Fri Dec 15 04:10:46 +0000 2023
1,elektabilitas tembus 45 persen ahy prabowo bis...,Iza_Zot,Fri Dec 15 04:10:46 +0000 2023
2,beri pujian ke ahy prabowo pemimpin yang hebat...,Iza_Zot,Fri Dec 15 04:10:43 +0000 2023
3,jokowi hebat setuju tapi melihat penampilan pr...,romobing,Fri Dec 15 04:10:41 +0000 2023
4,prabowo prediksi ahy akan jadi pemimpin di tin...,Iza_Zot,Fri Dec 15 04:10:38 +0000 2023
...,...,...,...
498,memang tidak mudah sekarang masyarakat mau tau...,HAPOSANM,Fri Dec 15 03:56:02 +0000 2023
499,visi prabowo rekan prabowo membawa harapan bar...,istimnatiara,Fri Dec 15 03:56:01 +0000 2023
500,pembenaran akan selalu ada tapi jika anda mau ...,DylanBr03197352,Fri Dec 15 03:56:01 +0000 2023
501,prabowo rekan prabowo berkomitmen untuk mengop...,clara144495,Fri Dec 15 03:56:00 +0000 2023


In [ ]:
#tokenize
tokenized = df['full_text'].apply(lambda x:x.split())
tokenized

0      [eh, botak, bilang, capres, dulu, ngobrol, ama...
1      [elektabilitas, tembus, 45, persen, ahy, prabo...
2      [beri, pujian, ahy, prabowo, pemimpin, hebat, ...
3      [jokowi, hebat, setuju, melihat, penampilan, p...
4      [prabowo, prediksi, ahy, jadi, pemimpin, tingk...
                             ...                        
498    [memang, mudah, sekarang, masyarakat, mau, tau...
499    [visi, prabowo, rekan, prabowo, membawa, harap...
500    [pembenaran, selalu, tapi, mau, merunut, kata,...
501    [prabowo, rekan, prabowo, berkomitmen, mengopt...
502    [program, program, prabowo, rekan, prabowo, je...
Name: full_text, Length: 451, dtype: object

In [ ]:
#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  return d_clean

tokenized = tokenized.apply(stemming)
tokenized.to_csv("preProcessingPrabowo.csv", index=False)

In [ ]:
data = pd.read_csv('preProcessingPrabowo.csv')

,full_text
0,eh botak bilang capres dulu ngobrol ama bahas ...
1,elektabilitas tembus 45 persen ahy prabowo men...
2,beri puji ahy prabowo pimpin hebat bangsa indo...
3,jokowi hebat tuju lihat tampil prabowo debat 1...
4,prabowo prediksi ahy jadi pimpin tingkat nasio...
...,...
95,pak prabowo pecat nih adminnya koq malah promo...
96,prabowo rekan prabowo punya trik khusus pilih ...
97,ayo kt undang amp pandu sorak utk ikut hadir t...
98,5 tempat wisata keluarga bandung rekomendasi l...
